In [3]:
import pandas as pd
from itertools import cycle, islice
import numpy as np
import random

In [ ]:
#input_text=open('/content/sample_text.txt', encoding="utf-8").readlines()
output_text_0=open('/content/human_evluation_0.txt', encoding="utf-8").readlines()
output_text_1=open('/content/T5-base-high_SARI-generations.txt', encoding="utf-8").readlines()
ROUTE = '/content/human_evaluation_form_'

In [ ]:
def unison_shuffled_copies(a, b, c):
    assert len(a) == len(b)
    assert len(a) == len(c)
    d = list(zip(a, b, c))
    random.shuffle(d)
    a, b, c = zip(*d)
    return a, b, c

def list_cycle(index,mylist): #List slicing with cycle
    temp = list(islice(cycle(mylist), 100))
    return temp[index*20:(index+2)*20]

def create_dataframe(index):
    my_frame=[]
    temp_input = list_cycle(index,input_text)
    temp_output0 = list_cycle(index,output_text_0)
    temp_output1 = list_cycle(index,output_text_1)
    temp_input, temp_output0, temp_output1 = unison_shuffled_copies(temp_input, temp_output0, temp_output1) #Randomize order

    #Randomize position
    for i in range(0,len(temp_input)):

        if random.randint(0,1) == 0:
            my_frame.append([temp_input[i], temp_output0[i]])
            my_frame.append([temp_input[i], temp_output1[i]])
        else:
            my_frame.append([temp_input[i], temp_output1[i]])
            my_frame.append([temp_input[i], temp_output0[i]])
    return my_frame


In [ ]:
assert len(input_text) == len(output_text_0)
assert len(input_text) == len(output_text_1)


for i in range(0,4):
    df = pd.DataFrame(create_dataframe(i))
    df.columns = ['complex','simple']
    df.fillna('',inplace=True)
    df.reset_index(inplace=True)
    wf = df.groupby(['complex','simple']).agg([np.mean])
    print(wf)
    with pd.ExcelWriter(f'{ROUTE}{i}.xlsx',mode='w') as writer:
        wf.to_excel(writer,index=True,header=False,sheet_name = str(i))


In [2]:
import os
from statistics import mean

In [ ]:

score = {'Strongly Disagree': 0, 'Disagree': 1, 'Neutral': 2, 'Agree': 3, 'Strongly Agree': 4}
output_0_mean=[]
output_0_simp=[]
output_1_mean=[]
output_1_simp=[]

def readScore():

    for i in range(0,4):
        source=[]
        path = f'/content/Human_evaluation_Form{i}.xlsx'
        if os.path.exists(path):
            df = pd.read_excel(path,skiprows=4)
            #print(df.columns.to_list)
            output = df['Simplified sentences'].to_list()
            meaning = df['Meaning preservation'].to_list()
            simplisity = df['Simplisity'].to_list()
            for j in range(0,80):
                simp_score = score[simplisity[j]]
                mean_score = score[meaning[j]]
                sentence = output[j]
                if sentence in output_text_0:
                    output_0_mean.append(mean_score)
                    output_0_simp.append(simp_score)
                    source.append('Control Token')
                elif sentence in output_text_1:
                    output_1_mean.append(mean_score)
                    output_1_simp.append(simp_score)
                    source.append('T5-base')
            print(source)
            with pd.ExcelWriter(f'/content/Source{i}.xlsx',mode='w') as writer:
                df = pd.DataFrame(source)
                df.to_excel(writer,index=True,header=False,sheet_name = str(i))
readScore()
print(mean(i for i in output_0_mean if i is not None))
print(mean(i for i in output_0_simp if i is not None))
print(mean(i for i in output_1_mean if i is not None))
print(mean(i for i in output_1_simp if i is not None))


['Control Token', 'T5-base', 'T5-base', 'Control Token', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'Control Token', 'T5-base', 'T5-base', 'Control Token', 'Control Token', 'T5-base', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'T5-base', 'Control Token', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'T5-base', 'Control Token', 'Control Token', 'T5-base', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'T5-base', 'Control Token', 'Control Token', 'T5-base', 'Control Token', 'T

In [ ]:
from re import search
source = open('/content/Source/PLABA_test_data.txt', encoding="utf-8").readlines()
target = open('/content/Source/final_gens_t5.txt', encoding="utf-8").readlines()
left_source = open('/content/Source/left_sentences.txt', encoding="utf-8").readlines()
left_target = open('/content/Source/left_gens_t5.txt', encoding="utf-8").readlines()

for i in range(1,41):
    for j in range(1,11):
        tgt=[]
        src = open(f'/content/Origin/Q{i}_A{j}.src.txt','r',encoding='utf-8').readlines()
        for k in src:
            sentence = k.strip('\n') + '\n'
            if sentence in source:
                tgt.append(target[source.index(sentence)])
            elif sentence in left_source:
                tgt.append(left_target[left_source.index(sentence)])
            else:
                print('warning!!!!!')
                print(k)
        tgt[-1] = tgt[-1].rstrip()
        with open(f'/content/drive/MyDrive/run1/Q{i}_A{j}.tgt.txt','w',encoding='utf-8') as w:
            w.writelines(tgt)

In [1]:
!pip install krippendorff

In [8]:
import krippendorff

In [14]:
# inter annotators agreement level
from sklearn.metrics import cohen_kappa_score

score = {'Strongly Disagree': 1, 'Disagree': 2, 'Neutral': 3, 'Agree': 4, 'Strongly Agree': 5}
raters = {'rater0':{},'rater1':{},'rater2':{},'rater3':{}}

def map_fun(a):
    return score[a]

def ordinal(score,source):
    assert type(score) is list
    assert type(source) is list
    if score[0]==score[1]:
        return 1 #Tie
    elif score[0] > score[1]:
        if source[0] == 'Control Token':
            return 2 #Control Token is better
        elif source[0] == 'T5-base':
            return 3 #T5-base is better
    elif score[0] < score[1]:
        if source[0] == 'Control Token':
            return 3 #T5-base is better
        elif source[0] == 'T5-base':
            return 2 #Control Token is better

def interval(score,source):
    assert type(score) is list
    assert type(source) is list
    if source[0] == 'Control Token': #Control token first
        return {source[0]:score[0],source[1]:score[1]}
    else:
        return {source[1]:score[1],source[0]:score[0]}

def get_score():
    for i in range(0,4):
            path = f'/content/Human_evaluation_Form{i}.xlsx'
            assert os.path.exists(path)
            df = pd.read_excel(path,skiprows=4)
            #print(df.columns.to_list)
            input = df['Orignal Text'].to_list()
            meaning = list(map(map_fun, df['Meaning preservation'].to_list()))
            simplisity = list(map(map_fun, df['Simplisity'].to_list()))
            source = df['Unnamed: 4'].to_list()
            #print(meaning)
            for j in range(0,40):
                #raters[f'rater{i}'][input[j*2]]={'meaning':ordinal(meaning[2*j:2*j+2],source[2*j:2*j+2]),'simplisity':ordinal(simplisity[2*j:2*j+2],source[2*j:2*j+2])}
                raters[f'rater{i}'][input[j*2]]={'meaning':interval(meaning[2*j:2*j+2],source[2*j:2*j+2]),'simplisity':interval(simplisity[2*j:2*j+2],source[2*j:2*j+2])}
get_score()

def get_cohen_kappa():
    temp_list = list(raters.keys())
    for i in temp_list:
        temp_list.remove(i)
        for j in temp_list:
            common_keys = set(raters[i].keys()) & set(raters[j].keys())
            if len(common_keys) == 20:
                rater0_meaing = []
                rater0_simplicity = []
                rater1_meaing = []
                rater1_simplicity = []
                for k in common_keys:
                    rater0_meaing.append(raters[i][k]['meaning'])
                    rater0_simplicity.append(raters[i][k]['simplisity'])
                    rater1_meaing.append(raters[j][k]['meaning'])
                    rater1_simplicity.append(raters[j][k]['simplisity'])
                print(f'Cohen Kappa of meaing preservation between {i} and {j} is {cohen_kappa_score(rater0_meaing, rater1_meaing, labels=[1, 2, 3])}')
                print(f'Cohen Kappa of simplicity between {i} and {j} is {cohen_kappa_score(rater0_simplicity, rater1_simplicity, labels=[1, 2, 3])}')

temp_list = list(raters.keys())
for i in temp_list:
        for j in temp_list:
            common_keys = set(raters[i].keys()) & set(raters[j].keys())
            if len(common_keys) == 20:
                rater0_meaning_CT = []
                rater0_simplicity_CT = []
                rater1_meaning_CT = []
                rater1_simplicity_CT = []
                rater0_meaning_T5 = []
                rater0_simplicity_T5 = []
                rater1_meaning_T5 = []
                rater1_simplicity_T5 = []
                for k in common_keys:
                    rater0_meaning_T5.append(raters[i][k]['meaning']['T5-base'])
                    rater0_simplicity_T5.append(raters[i][k]['simplisity']['T5-base'])
                    rater1_meaning_T5.append(raters[j][k]['meaning']['T5-base'])
                    rater1_simplicity_T5.append(raters[j][k]['simplisity']['T5-base'])

                    rater0_meaning_CT.append(raters[i][k]['meaning']['Control Token'])
                    rater0_simplicity_CT.append(raters[i][k]['simplisity']['Control Token'])
                    rater1_meaning_CT.append(raters[j][k]['meaning']['Control Token'])
                    rater1_simplicity_CT.append(raters[j][k]['simplisity']['Control Token'])
                print(f'Krippendorff\'s alpha of meaning preservation between {i} and {j} on T5-base is ', krippendorff.alpha(reliability_data=[rater0_meaing_T5,rater1_meaing_T5]))
                print(f'Krippendorff\'s alpha of simplicity between {i} and {j} on T5-base is ', krippendorff.alpha(reliability_data=[rater0_simplicity_T5,rater1_simplicity_T5]))
                print(f'Krippendorff\'s alpha of meaning preservation between {i} and {j} on Control Token is ', krippendorff.alpha(reliability_data=[rater0_meaing_CT,rater1_meaing_CT]))
                print(f'Krippendorff\'s alpha of simplicity between {i} and {j} on Control Token is ', krippendorff.alpha(reliability_data=[rater0_simplicity_CT,rater1_simplicity_CT]))
                #print(f'Cohen Kappa of simplicity between {i} and {j} is {cohen_kappa_score(rater0_simplicity, rater1_simplicity, labels=[1, 2, 3])}')

Cohen Kappa of meaning preservation between rater0 and rater1 on T5-base is  -0.05602240896358546
Cohen Kappa of simplicity between rater0 and rater1 on T5-base is  0.38550420168067234
Cohen Kappa of meaning preservation between rater0 and rater1 on Control Token is  0.06545879602571592
Cohen Kappa of simplicity between rater0 and rater1 on Control Token is  0.05220522052205223
Cohen Kappa of meaning preservation between rater0 and rater2 on T5-base is  -0.05602240896358546
Cohen Kappa of simplicity between rater0 and rater2 on T5-base is  0.20227272727272738
Cohen Kappa of meaning preservation between rater0 and rater2 on Control Token is  0.06545879602571592
Cohen Kappa of simplicity between rater0 and rater2 on Control Token is  0.007827788649706457
Cohen Kappa of meaning preservation between rater1 and rater0 on T5-base is  -0.05602240896358546
Cohen Kappa of simplicity between rater1 and rater0 on T5-base is  0.38550420168067234
Cohen Kappa of meaning preservation between rater1 a

*    *    *    *    *    3    4    1    2    1    1    3    3    *    3
1    *    2    1    3    3    4    3    *    *    *    *    *    *    *
*    *    2    1    3    4    4    *    2    1    1    3    3    *    4

[[nan, nan, nan, nan, nan, 3, 4, 1, 2, 1, 1, 3, 3, nan, 3], [1, nan, 2, 1, 3, 3, 4, 3, nan, nan, nan, nan, nan, nan, nan], [nan, nan, 2, 1, 3, 4, 4, nan, 2, 1, 1, 3, 3, nan, 4]]
